In [85]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
import pymysql

In [86]:
import os
import gc
from IPython.display import clear_output

In [87]:
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split

In [89]:
def create_model(X_train):
    model = keras.models.Sequential(
        [
            keras.layers.Dense(1024,activation="relu",input_shape=(X_train.shape[1],)),
            keras.layers.Dropout(0.2),
            keras.layers.Dense(512,activation="relu"),
            keras.layers.BatchNormalization(),
            keras.layers.Dense(256,activation="relu"),
            keras.layers.Dense(128),
            keras.layers.Dropout(0.2),
            keras.layers.Dense(64),
            keras.layers.BatchNormalization(),
            keras.layers.Dense(32),
            keras.layers.Dense(1)
        ]
    )
    
    model.summary()
    return model

In [90]:
data = pd.read_csv("refined_data.csv")

In [91]:
data_locations = np.unique(data.LOC)

In [92]:
for location in data_locations:
    print(location, "모델 작업중...")
    
    data = pd.read_csv("refined_data.csv")
    data = data.iloc[:,1:]
    data = data.drop([378451,378809,379999,291246,141024])
    data = data.loc[(data.SI>0) &(data.value != 0)].loc[data.LOC==location][["WS","TA","TD","HM","PV","VS","SS","SI","TS","value"]]
    value_mean = data.value.mean()
    
    X_raw = data.iloc[:,:-1]
    y_raw = data.iloc[:,-1]
    X_, X_test, y_, y_test = train_test_split(X_raw, y_raw, test_size=0.2)
    X_train, X_validation, y_train, y_validation = train_test_split(X_, y_, test_size=0.2)
    
    del(X_, y_, X_raw, y_raw, data)
    gc.collect()
    
    if os.path.isfile(f'./testModels/{location}_model.h5'):
        model = keras.models.load_model(f'./testModels/{location}_model.h5',compile=False)
        print("\n model +1 강 \n")
    else:
        model = create_model(X_train)
        print("\n model 제작 \n")
        print()

    model.compile(optimizer="adam", loss="mean_absolute_error")
    check_point_cb = keras.callbacks.ModelCheckpoint(f'./testModels/{location}_model.h5', save_best_only=True)
    model.fit(X_train, y_train, epochs=20, validation_data=(X_validation, y_validation), callbacks=[check_point_cb])
    clear_output()
    
    print("="*50)
    print(location, "End fitting")
    print("="*50)
    model.evaluate(X_test,y_test)
    print("비교용 평균치:",value_mean)
    print()
    

충북 End fitting
97/97 [==============================] - 0s 2ms/step - loss: 14311546.0000
비교용 평균치: 68064804.86385848

